<a href="https://colab.research.google.com/github/srinisuryadevara/Course/blob/master/FoodRecSys_Train_Test_Split_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
import os
# give googe drive the required permission
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Create a folder in your drive and add the name of that folder here.
# For example, for the code below to run correctly, you need to have a folder named FoodRecSys in 'My Drive'.
# The said folder will be your home directory for the rest of the project.
# You will be able to save and read data from the folder.

os.chdir("/content/drive/MyDrive/FoodRecSys/")
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1sSKpoSGnu5kt4yDE4V77EZp7JWN-SUmH/food_recsys_project/Code Files'

In [ ]:
os.chdir("/content/drive/MyDrive/food_recsys_project/Code Files/")
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1sSKpoSGnu5kt4yDE4V77EZp7JWN-SUmH/food_recsys_project/Code Files'

In [ ]:
try:
  import pyspark
except:
  !pip install pyspark==3.1.2
  import pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 212.4 MB 72 kB/s 
     |████████████████████████████████| 198 kB 13.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880769 sha256=072c650ae50cde8052dcd6ed947132078af197bcd430b2eafb74174aff75ecc1
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

In [ ]:
spark = SparkSession.builder.master("local").config('spark.ui.port', '4050').getOrCreate()

In [ ]:
spark

## Imports

In [ ]:
import pandas as pd
import numpy as np

from pyspark.sql import functions as F
# Import for typecasting columns
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType,StringType, ArrayType

## Read the data

In [ ]:
raw_ratings_df = (spark.read.csv("raw_ratings_small.csv", # modify the path to read the data
                                 header=True,
                                 inferSchema= True))

In [ ]:
raw_recipes_df = spark.read.csv("raw_recipes_small.csv", # modify the path to read the data
                                header=True,
                                inferSchema=True)

In [ ]:
assert (raw_recipes_df.count(), len(raw_recipes_df.columns)) == (20340, 13)
assert (raw_ratings_df.count(), len(raw_ratings_df.columns)) == (93357, 5)

#### Decide a split date based on the ratings dataframe.

In [ ]:
# Find the number of data points in the interaction dataset.
# You can use the count() method.
# The output must be an integer.

num_review_int = raw_ratings_df.count()

#### Task 01 - Train Test Split

Divide the data into train and test based on the 80 - 20 split using the approach discussed. You will have to save the data in a parquet file.

In [ ]:
test_num_reviews_int = round(num_review_int *0.2)

In [ ]:
# Sort the interactions dataset in descending order of review date.
# Extract ```test_num_reviews_int``` most recent reviews.

temp_ratings_df = (raw_ratings_df.sort("review_date", ascending=False)
                                 .limit(test_num_reviews_int)
                  )

In [ ]:
assert temp_ratings_df.count()  == 18671
assert raw_recipes_df.collect()[11][4] <= raw_recipes_df.collect()[10][4]

In [ ]:
temp_ratings_df.collect()[-1][4]

'2011-07-17'

Split the data into two parts before and after 2011-07-17.

- All reviews in the ratings data after 2011-07-17 will not exsist in the training set.
- For all future predictions the date will be set at 2011-07-18.   

In [ ]:
# Join raw_recipes and raw_ratings
# Use recipe_id as the key to join these dataframes
# The resulting dataframe must have all rows from the raw_ratings dataframe.

interaction_level_df = raw_ratings_df.join(
                                           # dataframe 2,
                                           # key to join
                                           # how to join
                                          )

In [ ]:
# Use the filter command to separate the datasets.
# All interactions which were rated BEFORE '2011-07-17' will be train data.

train_interaction_level_df  = (interaction_level_df.filter(
                                             # add code to filter
                                           )))

In [ ]:
# Use the filter command to separate the datasets.
# All interactions which were rated ON OR AFTER '2011-07-17' will be test data.

test_interaction_level_all_recipies_df  = (interaction_level_df.filter(
                                             # add code to filter
                                           )))

In [ ]:
assert (test_interaction_level_all_recipies_df.count(), len(test_interaction_level_all_recipies_df.columns)) == (18684, 18)
assert (train_interaction_level_df.count(), len(train_interaction_level_df.columns)) == (74673, 18)

In [ ]:
# create data files for modeling

(train_interaction_level_df.coalesce(1)
                           .write.mode('overwrite')
                           .parquet('train_interaction_level_df.parquet'))  # change the file name and file path

(test_interaction_level_all_recipies_df.coalesce(1)
                                       .write.mode('overwrite')
                                       .parquet('test_interaction_level_df.parquet'))  # change the file name and file path